<a href="https://colab.research.google.com/github/WaymentSteeleLab/Dyna-1/blob/main/colab/Dyna_1_ESM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="" height="200" align="right" style="height:240px">

##Dyna-1 (ESM-2)
Easy-to-use dynamics prediction with [Dyna-1](https://www.biorxiv.org/content/10.1101/2025.03.19.642801v1). Dyna-1 predicts the probability that each residue  experiences millisecond motions. We provide this Google Colab using a pre-trained version of ESM-2 to be available under a Non-Commercial License. <i>This was not the final reported model.</i>
<br><br>

A Google Colab for the best performing version of Dyna-1 is available at this [Google Colab](https://colab.research.google.com/github/WaymentSteeleLab/Dyna-1/blob/main/colab/Dyna_1.ipynb). The model requires the pre-trained wieghts from [ESM-3](https://www.biorxiv.org/content/10.1101/2024.07.01.600583v1), is subject to the License terms of ESM-3, and requires read permission to the [esm3-sm-open-v1](https://huggingface.co/EvolutionaryScale/esm3-sm-open-v1) weights.
<br><br>


<i>This colab was written by Gina El Nesr (gelnesr@stanford.edu)</i>

In [ ]:
#@title **IMPORTANT:** run this cell first before doing 'Runtime →> Run all'
#@markdown - The latest update to Google Colab broke numpy; this is a temporary patch.
#@markdown - Note after running this cell, the session will crash (this is normal).

import os, numpy, signal

if numpy.__version__ != '1.26.4':
  print(f"Current numpy version {numpy.__version__} is incorrect. Installing 1.26.4...")
  os.system("'pip uninstall -y numpy")
  os.system("pip install numpy==1.26.4")
  # Restart the runtime using os.kill
  os.kill(os. getpid(), signal.SIGKILL)
else:
  print ("Numpy version is correct (1.26.4)")

In [ ]:
#@title Install dependencies & download model weights (~3 min)
os.system('git clone https://github.com/WaymentSteeleLab/Dyna-1.git --depth 1')

print('installing requirements for Dyna-1...')
import os
os.system('pip install -r Dyna-1/requirements.txt')
os.system('pip install gdown, py3Dmol, torcheval')
print('requirements installed!')

import gdown
if not os.path.isfile('Dyna-1/model/weights/dyna1.pt'):
  print('downloading model weights...')
  os.system('pip install -U huggingface_hub[cli]')
  os.system('huggingface-cli download gelnesr/Dyna-1 --local-dir "/content/Dyna-1/model/weights/"')
print('model weights downloaded!')
if not os.path.exists('/content/outputs'):
  os.mkdir('/content/outputs')
if not os.path.exists('/content/inputs'):
  os.mkdir('/content/inputs')

In [ ]:
#@title Prepare Input
from google.colab import files
import os
import re
import hashlib
import random
import sys

#@markdown ####Dyna-1 using ESM-2 requires a sequence input. Please specify below.
sequence = 'MQYKLVINGKTLKGETTTKAVDAETAEKAFKQYANDNGVDGVWTYDDATKTFTVTE' #@param {type:"string"}

if sequence == '':
  sys.exit('No sequence given.')

alphabets = {'protein': re.compile('^[acdefghiklmnpqrstvwy]*$', re.I)}
if alphabets['protein'].search(sequence) is None:
  sys.exit('Invalid sequence given.')

In [ ]:
#@title Model Parameters
job_name = 'test' #@param {type:"string"}
if job_name == '':
  name = random.randint(0, 100000)
  job_name = f'{name}-Dyna1_ESM2'

In [ ]:
#@title Run Dyna-1 (ESM-2) (~30s to 2 min)
import os
if os.getcwd() != '/contents/Dyna-1':
    os.chdir('/content/Dyna-1')

import warnings
warnings.filterwarnings("ignore")
import utils
import torch
import random
import argparse
import numpy as np
import pandas as pd

from model.model import *
from transformers import AutoTokenizer

try:
    sequence
except NameError:
    sys.exit('No sequence found.')

warnings.filterwarnings("ignore")
DEVICE = torch.device("cpu")

if torch.cuda.is_available():
   DEVICE = torch.device("cuda:0")

model = ESM_model(method='esm2', nheads=8, nlayers=12, layer=30).to(DEVICE)
model.load_state_dict(torch.load('model/weights/dyna1-esm2.pt', map_location=DEVICE), strict=False)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(f"facebook/esm2_t6_8M_UR50D")
seq_input = tokenizer.encode(sequence, add_special_tokens=False, return_tensors='pt').to(DEVICE)
sequence_id = seq_input != 0

logits = model(seq_input, sequence_id)
p = utils.prob_adjusted(logits).cpu().detach().numpy()

out_df = pd.DataFrame({'position': np.arange(1,len(p)+1), 'residue': np.array(list(sequence)), 'probs': p,})
out_df.to_csv(f'/content/outputs/{job_name}.csv', index=False)

In [ ]:
#@title Plot probabilities
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
if os.path.isfile(f'/content/outputs/{job_name}.csv'):
  data = pd.read_csv(f'/content/outputs/{job_name}.csv')
else:
  sys.exit('No files outputted.')
probs = data['probs']

# Plot the array
plt.figure(figsize=(10,5), dpi=200)
plt.plot(np.arange(1, len(probs)+1, 1), probs)
plt.ylim(0, 1)
plt.ylabel('Dyna-1 P(exchange)')
plt.title(f'{job_name}')
plt.show()